# Preparando o Ambiente Virtual

In [1]:
# Instalando o cartopy
!pip uninstall -q shapely --yes
!pip install -q shapely cartopy --no-binary shapely --no-binary cartopy

# Instalando Py-art
!pip install -q arm_pyart

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.4/283.4 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 96.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 97.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.9/131.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━

In [2]:
# Montando o Drive
from google.colab import drive
drive.mount('/content/drive')
dir =  '/content/drive/MyDrive/2-PESQUISA/1-ORIENTACOES/TFG_MAYCON/dados_radar_fcth_2024-01-22/'

Mounted at /content/drive


# Plota figura

In [ ]:
# Importa bibliotecas
import pyart # necessário so para a barra de cor
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
import cartopy
import glob
import os

# Informações de tamanho da matriz e CENTRO do radar
nx, ny = 480, 480
lon_radar, lat_radar = -45.972790, -23.600795

# Sistema de coordenadas
crs = ccrs.Orthographic(central_latitude=lat_radar, central_longitude=lon_radar)

# Loop dos arquivos
for arq in sorted(glob.glob(f'{dir}input_dados_descompactados/V2_PNOVA2_DPSRI-H030-R1000-dBR_*.PROD')):

    print('PROCESSANDO === >>>', arq)

    # Leitura do arquivo binário
    f = open(arq, 'rb')
    data = np.fromfile(f, count=nx*ny, dtype=np.float32)
    f.close()

    # Aplica reshape na matriz de dados
    cappi = np.reshape(data, [nx, ny], order='C')

    # Inverte o eixo y
    cappi = np.flip(cappi, 0)

    # Elimina a achuva < 1 mm/h
    cappi[cappi < 1] = np.nan

    # Coordenadas
    x = np.arange(nx)*1000 - 240000
    y = x
    xx, yy = np.meshgrid(x,y)

    # Extrai a data e horário
    basename = os.path.basename(os.path.splitext(arq)[0])
    ano, mes, dia, hor, min = basename[31:35], basename[35:37], basename[37:39], basename[39:41], basename[41:43]

    # Título da figura
    titulo = f' Preciptação 3 km - FCTH: {ano}-{mes}-{dia} {hor}{min} UTC'

    # formato da figura
    fig = plt.figure(figsize=[8,6])
    ax=fig.add_subplot(1,1,1, projection=crs)

    #plota o mapa, rios, litoral, etc
    ax.add_feature(cartopy.feature.LAND)
    #ax.add_feature(cartopy.feature.OCEAN)
    #ax.add_feature(cartopy.feature.LAKES, alpha=0.5)
    #ax.add_feature(cartopy.feature.RIVERS)
    ax.add_feature(cartopy.feature.COASTLINE, linewidth=0.5, color='black')
    ax.add_feature(cartopy.feature.BORDERS, linestyle=':', linewidth=0.3)

    # Extensão da imagem
    ax.set_extent((-240000, 240000, -240000, 240000), crs=crs)

    # Plota os círculos de distancia de 100 e 240km
    circ1=plt.Circle((0, 0), radius=100000, color='grey', fill=False, lw=0.5)
    circ2=plt.Circle((0, 0), radius=240000, color='grey', fill=False, lw=0.5)
    ax.add_patch(circ1)
    ax.add_patch(circ2)

    # Plota o grid
    gl = ax.gridlines(draw_labels=True, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False

    # Plota mapa
    vmin = 0
    vmax = 170

    levels = np.arange(vmin,vmax, 10)

    rain = ax.contourf(xx,
                       yy,
                       cappi,
                       levels=levels,
                       vmin=vmin, vmax=vmax,
                       cmap='LangRainbow12',
                       transform=crs)

    # Plota a barra de cor
    plt.colorbar(rain, label='[mm/h]', pad=0.01)
    plt.title(titulo, fontsize='x-large')

    # Salva figura
    #plt.show()
    plt.tight_layout()
    plt.savefig(f'{dir}output/cappi_rain_fcth_{ano}{mes}{dia}_{hor}{min}.png')
    plt.close()


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119

PROCESSANDO === >>> /content/drive/MyDrive/2-PESQUISA/1-ORIENTACOES/TFG_MAYCON/dados_radar_fcth_2024-01-22/input_dados_descompactados/V2_PNOVA2_DPSRI-H030-R1000-dBR_2025012200000500dBR.dpsri.PROD
